In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import geopy.distance

In [35]:
df = pd.read_csv('../data/raw/data_real_estate_2018_2021.csv')
df.shape

(5477006, 13)

In [36]:
df.head()

,price,date,time,geo_lat,geo_lon,region,building_type,level,levels,rooms,area,kitchen_area,object_type
0,6050000,2018-02-19,20:00:21,59.805808,30.376141,2661,1,8,10,3,82.6,10.8,1
1,8650000,2018-02-27,12:04:54,55.683807,37.297405,81,3,5,24,2,69.1,12.0,1
2,4000000,2018-02-28,15:44:00,56.295250,44.061637,2871,1,5,9,3,66.0,10.0,1
3,1850000,2018-03-01,11:24:52,44.996132,39.074783,2843,4,12,16,2,38.0,5.0,11
4,5450000,2018-03-01,17:42:43,55.918767,37.984642,81,3,13,14,2,60.0,10.0,1


In [37]:
### Определим значение наших столбцов
#  `date` - дата публикации объявления; 
#  `time` - время публикаци объявления; 
#  `geo_lat` - значение координаты (широта (град));
#  `geo_lon` - значение координаты (долгота (град));
#  `region` - код региона РФ;
#  `building_type` - Тип здания. 0 - Прочее. 1 - Панельный дом. 2 - Монолит. 3 - Кирпичный. 4 - Блочный. 5 - Деревянный; 
#  `object_type` - Тип квартиры. 1 - Вторичное жилье; 11 - Новая квартира в новостройке;
#  `level` - Этаж, на котором находится квартира; 
#  `levels` - Количество этажей;
#  `rooms` - Количество жилых комнат. Если значение -1 - это значит, что квартира является "студией";
#  `area` - Совокупная площадь квартиры;
#  `kitchen_area` - Площадь кухни; 
#  `price` - Цена в рублях РФ.

In [38]:
REGION_ID = 3  # City of Saint Petersburg 2661, Moscow = 3 , Moscow oblast = 81

MIN_AREA = 20  # Outlier range for floor area
MAX_AREA = 120

MIN_ROOMS = 1
MAX_ROOMS = 4

MIN_KITCHEN = 5  # Outlier range for kitchen area
MAX_KITCHEN = 30

MIN_PRICE = 2_000_000  # Outlier range for price
MAX_PRICE = 50_000_000

In [74]:
def clean_data(df: pd.DataFrame) -> pd.DataFrame:
    """Function removes excess columns and enforces correct data types.

    :param df: Original DataFrame
    :return: Updated DataFrame

    """
    # Column actually contains -1 and -2 values presumably for studio apartments.
    df['timestamp'] = pd.to_datetime(df['date'])
    df['target'] = df['price'].abs()  # Fix negative values
    # Drop price and area outliers.
    df= df[(df['rooms'] <= MAX_ROOMS) & (df['rooms'] >= MIN_ROOMS) ]
    df = df[(df['area'] <= MAX_AREA) & (df['area'] >= MIN_AREA)]
    df = df[(df['price'] <= MAX_PRICE) & (df['price'] >= MIN_PRICE)]
    
    return df


def select_region(df: pd.DataFrame) -> pd.DataFrame:
    """Function selects the listings belonging to a specified region.
    
    :param df: Original DataFrame with all listings
        
    :return: Filtered DataFrame

    """
    df = df[ df['region'] == REGION_ID]  
    df.drop('region', axis=1, inplace=True)
    print(f'Selected {len(df)} samples in region {REGION_ID}.')
    return df


def add_features(df: pd.DataFrame) -> pd.DataFrame:
    # Replace "date" with numeric features for year and month.
    df['year'] = df['timestamp'].dt.year
    df['month'] = df['timestamp'].dt.month
    #df.drop('date', axis=1, inplace=True)
    # Apartment floor in relation to total number of floors.
    moscow_centre = (55.751003, 37.617964)
    ms = np.concatenate([df.geo_lat.values.reshape(-1,1), df.geo_lon.values.reshape(-1,1)], axis = 1)
    dist_msk = list(map(lambda x: geopy.distance.geodesic(x, moscow_centre).km, ms))   
    df['distance_from_centre'] = dist_msk
    # удаляем выбросы по расстоянию
    df = df[df.distance_from_centre < 25]
    df.drop('distance_from_centre', axis=1, inplace=True)
    df.reset_index(drop = True, inplace = True)
    return df

def drop_field_for_clear(df : pd.DataFrame) -> pd.DataFrame:
    df.drop('time', axis=1, inplace=True)
    df.drop('date', axis=1, inplace=True)
    df.drop('geo_lat', axis=1, inplace=True)
    df.drop('geo_lon', axis=1, inplace=True)
    df.drop('building_type', axis=1, inplace=True)
    df.drop('level', axis=1, inplace=True)
    df.drop('levels', axis=1, inplace=True)
    df.drop('area', axis=1, inplace=True)
    df.drop('kitchen_area', axis=1, inplace=True)
    df.drop('price', axis=1, inplace=True)
    return df

def drop_field_for_geo_clastering(df : pd.DataFrame) -> pd.DataFrame:
    df.drop('time', axis=1, inplace=True)
    df.drop('date', axis=1, inplace=True)
    df.drop('building_type', axis=1, inplace=True)
    df.drop('level', axis=1, inplace=True)
    df.drop('levels', axis=1, inplace=True)
    df.drop('area', axis=1, inplace=True)
    df.drop('kitchen_area', axis=1, inplace=True)
    df.drop('price', axis=1, inplace=True)
    df.drop('rooms', axis=1, inplace=True)
    df.drop('timestamp', axis=1, inplace=True)
    df.drop('object_type', axis=1, inplace=True)
    df.drop('year', axis=1, inplace=True)
    df.drop('month', axis=1, inplace=True)
    df.drop('target', axis=1, inplace=True)
    return df

In [42]:
msk_df = select_region(df.copy())
msk_df.shape

C:\Users\Xiaomi Air 13\AppData\Local\Temp\ipykernel_25044\239767866.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop('region', axis=1, inplace=True)


Selected 439511 samples in region 3.


(439511, 12)

In [43]:
msk_df

,price,date,time,geo_lat,geo_lon,building_type,level,levels,rooms,area,kitchen_area,object_type
6,4704280,2018-03-04,12:35:25,55.621097,37.431002,2,1,25,1,31.7,6.0,11
20,8000000,2018-03-17,06:46:32,55.738876,37.825370,1,5,9,2,45.0,6.0,1
29,11500000,2018-04-03,17:40:53,55.798921,37.738090,1,14,16,3,64.9,8.0,1
34,10200000,2018-04-07,15:23:20,55.655307,37.614605,2,7,14,2,52.8,10.0,1
44,11500000,2018-04-18,21:52:41,55.701330,37.507412,1,14,17,1,42.0,12.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
5476999,12850000,2021-05-01,20:13:47,55.701280,37.642654,2,12,24,1,41.0,9.0,1
5477001,19739760,2021-05-01,20:13:58,55.804736,37.750898,1,8,17,4,93.2,13.8,11
5477002,12503160,2021-05-01,20:14:01,55.841415,37.489624,2,17,32,2,45.9,6.6,11
5477004,11831910,2021-05-01,20:14:12,55.804736,37.750898,1,8,33,2,52.1,18.9,11


In [44]:
msk_df = clean_data(msk_df.copy())
msk_df.shape

(389721, 14)

In [45]:
msk_df = add_features(msk_df.copy())
msk_df.shape

C:\Users\Xiaomi Air 13\AppData\Local\Temp\ipykernel_25044\239767866.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop('distance_from_centre', axis=1, inplace=True)


(368158, 16)

In [46]:
msk_df.head()

,price,date,time,geo_lat,geo_lon,building_type,level,levels,rooms,area,kitchen_area,object_type,timestamp,target,year,month
0,4704280,2018-03-04,12:35:25,55.621097,37.431002,2,1,25,1,31.7,6.0,11,2018-03-04,4704280,2018,3
1,8000000,2018-03-17,06:46:32,55.738876,37.825370,1,5,9,2,45.0,6.0,1,2018-03-17,8000000,2018,3
2,11500000,2018-04-03,17:40:53,55.798921,37.738090,1,14,16,3,64.9,8.0,1,2018-04-03,11500000,2018,4
3,10200000,2018-04-07,15:23:20,55.655307,37.614605,2,7,14,2,52.8,10.0,1,2018-04-07,10200000,2018,4
4,11500000,2018-04-18,21:52:41,55.701330,37.507412,1,14,17,1,42.0,12.0,1,2018-04-18,11500000,2018,4


In [63]:
msk_clear = drop_field_for_clear(msk_df.copy())
msk_clear.shape

(368158, 6)

In [64]:
msk_clear.head()

,rooms,object_type,timestamp,target,year,month
0,1,11,2018-03-04,4704280,2018,3
1,2,1,2018-03-17,8000000,2018,3
2,3,1,2018-04-03,11500000,2018,4
3,2,1,2018-04-07,10200000,2018,4
4,1,1,2018-04-18,11500000,2018,4


In [65]:
msk_clear.to_csv(r'{0}.csv'.format('../data/interim/msk_2018_2021_clear'))

In [75]:
msk_geo = drop_field_for_geo_clastering(msk_df.copy())
msk_geo.shape

(368158, 2)

In [76]:
msk_geo.head()

,geo_lat,geo_lon
0,55.621097,37.431002
1,55.738876,37.825370
2,55.798921,37.738090
3,55.655307,37.614605
4,55.701330,37.507412


In [77]:
msk_geo.to_csv(r'{0}.csv'.format('../data/interim/msk_geodata'))